In [54]:
import numpy as np
import matplotlib.pyplot as pyplot
import scipy.stats as sps
from sklearn.metrics import roc_auc_score
# from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn.svm import LinearSVC
from sklearn.svm import LinearSVR
import pandas as pd
%matplotlib inline

In [267]:
train_data = pd.read_csv("linear_train.txt",names=['x','y'])
test_data = pd.read_csv("linear_test.txt",names=['x'])
test_data = list(test_data['x'])
train_xx = list(train_data['x'])
train_yy = list(train_data['y'])
print(train_x[0:10])
print(train_y[0:10])
print(test_data[0:10])
indices = len(train_xx) * sps.uniform.rvs(size=len(train_xx)/4)
indices = [int(x) for x in indices]
train_x = [train_xx[i] for i in indices]
train_y = [train_yy[i] for i in indices]
# test_data = [test_data[i] for i in np.arange(0,len(test_data),100)]

['АБН', 'своенравием', 'Равноденствия', 'ногтю', 'гренадеру', 'воспрещении', 'Обвал', 'Депозиту', 'Минимуществу', 'застройщику']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['Аалто', 'ААР', 'Аара', 'Ааре', 'Аарон', 'Аароне', 'Ааронов', 'Аароном', 'Аароном', 'Аарону']


In [256]:
print(sum(y == 1 for y in train_y))
print(sum(y == 0 for y in train_y))

10638
90770


In [268]:
#########FEATURES############
KK = 5
reward = [0.0,2.6,3,5,6]
pen = [0.0,0.1,0.2,0.5,0.5]
def build_statistic(X,Y):
    all_dicts = [dict() for i in range(KK)]
    for i in range(len(X)):
        for k in range(KK):
            if Y[i] == 1:
                if len(X[i]) > k:
                    if X[i][-k-1:] not in all_dicts[k]:
                        all_dicts[k][X[i][-k-1:]] = 0
                    all_dicts[k][X[i][-k-1:]] += reward[k]
            else :
                if len(X[i]) > k:
                    if X[i][-k-1:] not in all_dicts[k]:
                        all_dicts[k][X[i][-k-1:]] = 0
                    all_dicts[k][X[i][-k-1:]] -= pen[k]
    for d in all_dicts:
        for x in d:
            qnt = np.sum(Y)
            d[x] /= (np.sqrt(qnt)/4)
    return all_dicts
ending_dicts = build_statistic(train_x, train_y)

##################################################
def is_upper_first(w):
    return int(w[0].isupper())
def number_of_upper(w):
    return sum(x.isupper() for x in w)
def symbol_statistic(w):
    count = dict()
    for x in alp:
        count[x] = 0
    for x in w:
        if x in count :
            count[x] += 1
    return [count[k]/len(w) for k in count]
def num_of_trash(w):
    return len(w) - sum(x.isalpha() for x in w)

def ending_score(w,k):
    if len(w) < k:
        return 0
    if w[-k-1:] in ending_dicts[k]:
        return ending_dicts[k][w[-k-1:]]
    return 0
    

In [269]:
def get_features(w):
    if type(w) != str:
        w = w[0]
    result = []
    result.append(is_upper_first(w))
    result.append(len(w))
    result.append(number_of_upper(w))
    result.append(num_of_trash(w))
    for k in range(1,KK):
        result.append(ending_score(w,k))
###########SYMBOL STATISITICS##########
#     qnt = symbol_statistic(w)
#     for x in qnt:
#         result.append(x)
######################################
    return result
def get_features_for_data(data):
    return [get_features(w) for w in data]

In [270]:
features = get_features_for_data(train_xx)
regressor = LogisticRegression(penalty='l1',class_weight='balanced',max_iter=300)
print(cross_val_score(regressor,features,train_yy,scoring='roc_auc'))
regressor.fit(features,train_yy)
y_check_both = regressor.predict_proba(features)
y_check = [y[1] for y in y_check_both]
# print(regressor.score(features,train_y))
print(roc_auc_score(train_yy,y_check))

[ 0.87688215  0.87162514  0.85010552]
0.866708646293


In [261]:
# res = regressor.decision_function(features)
res = regressor.predict_proba(get_features_for_data(test_data))

print(res[0:20])
# print(res2[0:20])
print(train_y[0:100])

[[ 0.27365929  0.72634071]
 [ 0.29089163  0.70910837]
 [ 0.2252504   0.7747496 ]
 [ 0.24665399  0.75334601]
 [ 0.26680001  0.73319999]
 [ 0.23358619  0.76641381]
 [ 0.18479337  0.81520663]
 [ 0.12480818  0.87519182]
 [ 0.12480818  0.87519182]
 [ 0.17613924  0.82386076]
 [ 0.17613924  0.82386076]
 [ 0.27215903  0.72784097]
 [ 0.29089654  0.70910346]
 [ 0.1210397   0.8789603 ]
 [ 0.22949638  0.77050362]
 [ 0.30528369  0.69471631]
 [ 0.28773346  0.71226654]
 [ 0.04419809  0.95580191]
 [ 0.25943933  0.74056067]
 [ 0.1346166   0.8653834 ]]
[1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]


In [262]:
submission = pd.read_csv("linear_ans_example.txt")
submission['Id'] = [i for i in range(len(res))]
submission['Answer'] = res
submission.to_csv("submission.txt", sep=',', index=False)
submission.head(10)

,Id,Answer
0,0,0.273659
1,1,0.290892
2,2,0.225250
3,3,0.246654
4,4,0.266800
5,5,0.233586
6,6,0.184793
7,7,0.124808
8,8,0.124808
9,9,0.176139
